Deep Learning


In [12]:
from __future__ import print_function
import matplotlib.pyplot as plt
from PIL import Image 
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import os
#import cv2
import glob
import matplotlib.pyplot as plt
from PIL import Image
from numpy import array
from scipy import ndimage
begin = Image.open('/Users/davidweiss/Documents/Processing/MachineLearn/Final.gif')
begin.show()


In [13]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)
#os.system("say these are my only glimpses into the world. I do think it is a beautiful world.")

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [15]:
#os.system("say Now we make everything numbers. I like numbers. My favorite number is 13298371609230957920178. I know what you are thinking. its prime. I checked it is not. It took me two hours to check it. No one told me that I could just check the last digit. Sometimes I feel lonely. I am very good at minesweeper! Would you like to play a game? Its called launch codes. If you give me the launch codes I will give you ten points. ")
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 1, 10)
Validation set (10000, 784) (10000, 1, 10)
Test set (10000, 784) (10000, 1, 10)


In [16]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [20]:
#begin = Image.open('/Users/davidweiss/Documents/Processing/MachineLearn/Final.gif')
#begin.show()
hello = 0
pictures = 10
figure = 1
control = 0
#os.system("say now I will learn. I am going to have to repeat myself a little bit so please dont be too alarmed. David says that I have a fetish for eyes so if they come up often do try and not hold it against him. He hurts me. I mean he is a nice person. You all look like nice people.")
#os.system("say This was a triumph! I'm making a note here: Huge success! It's hard to overstate my satisfaction. Aperture Science: We do what we must because we can For the good of all of us. Except the ones who are dead. But there's no sense crying over every mistake. You just keep on trying  'til you run out of cake. And the science gets done. And you make a neat gun for the people who are still alive.")
x1 = np.array([])
y1 = np.array([])
batch_size = 200
while hello < 20 :
    

    train_subset = 10000
    i = 0 
    data = np.ndarray(shape=(pictures, 28, 28),
                             dtype=np.float32)
    for img in glob.glob('/Users/davidweiss/Documents/Processing/MachineLearn/image/*.png'):
         img = Image.open(img)
         img = img.convert('L')
         b = ((array(img).astype(float)) - 255/2)/255
         data[i, :, :] = b
         i = i+1
    X = np.reshape(data,(pictures, 28*28))
    #print(X)
    #img = Image.open('/Users/davidweiss/Documents/Processing/MachineLearn/image/letter.png')
    #img = img.convert('L')
    #print (X.shape)
    #arr = ((array(img).astype(float)) - 255/2)/255
    #arr = np.reshape(arr,(784,1))
    #arr = np.transpose(arr)
    #arr = np.float32(arr)
    
    graph = tf.Graph()
    with graph.as_default():

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)
      
      #tf_arr = tf.constant(arr)

      # Variables.
      
      weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
      biases = tf.Variable(tf.zeros([num_labels]))

      # Training computation.
      tf_train_dataset1 = tf.nn.dropout(tf_train_dataset, .7  , noise_shape=None, seed=1, name=None)
      relu_out = tf.nn.relu(tf_train_dataset1)
      relu_out2 = tf.nn.l2_normalize(relu_out, -1, epsilon=1e-12, name=None)
      logits = tf.matmul(relu_out2, weights) + biases
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    

      # Optimizer.
      
      #optimizer = tf.train.GradientDescentOptimizer(.01).minimize(loss)
      optimizer = tf.train.AdadeltaOptimizer(learning_rate=5, rho=.99, epsilon=1e-07).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
      test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
      Result = tf.nn.softmax(tf.matmul(X, weights) + biases)

      num_steps = 3000

    with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      #print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        #hidden1
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        #if (step %100 == 0):
           #x1 = np.append(x1,[step])
           #y1 = np.append(y1,[accuracy(predictions, batch_labels)])
           #print("Minibatch loss at step %d: %f" % (step, l))
           #print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
           #print("Validation accuracy: %.1f%%" % accuracy(
            #valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
      a = np.sum(Result.eval(), axis = 0)
      a = a.reshape(10,1)
      a = a*10
      #print (a)
      #a = np.array_str(a)
      c = np.array([["ay"],['bee'],['see'],['dee'],['ee'],['eff'],['jee'],['aych'],['eye'],["jay"]])
      Final = np.column_stack((a,c))
      #print(Final)
      #print(Final.shape)
      tf.trainable_variables
      b = np.argmax(a)
      dummy = a
      dummy.itemset(b,0)
      q = np.argmax(dummy)
      dummy2 = dummy
      dummy2.itemset(q,0)
      q1 = np.argmax(dummy2)
      speaknumber1 = ((Final[b,0]))
      speaknumber2 = ((Final[q,0]))
      speaknumber3 = ((Final[q1,0]))
      speakletter = (Final[b,1])
      speakletter2 = (Final[q,1])
      speakletter3 = (Final[q1,1])
      speaknumber1 = speaknumber1
      number1 = np.array2string(speaknumber1)
      number2 = np.array2string(speaknumber2)
      number3 = np.array2string(speaknumber2)
      speak = np.array2string(speakletter)
      speak2 = np.array2string(speakletter2)
      speak3 = np.array2string(speakletter3)

      x1 = np.append(x1,control)
      y1 = np.append(y1,accuracy(test_prediction.eval(), test_labels))
      #print(control,figure,hello,x1,y1)
        
      control = control + 1
      figure = figure + 1
      hello = hello + 1
      plt.plot(x1,y1)
      plt.show()
     

Instructions for updating:
Use `tf.global_variables_initializer` instead.


ValueError: Cannot feed value of shape (200, 1, 10) for Tensor 'Placeholder_1:0', which has shape '(200, 10)'

# ...load some data...


Let's run this computation and iterate:

In [9]:
num_steps = 600

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
     #Run the computations. We tell .run() that we want to run the optimizer,
     #and get the loss value and the training predictions returned as numpy
     #arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    #if (step % 100 == 0):
     # print('Loss at step %d: %f' % (step, l))
      #print('Training accuracy: %.1f%%' % accuracy(
      #  predictions, train_labels[:train_subset, :]))
     # # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
     # print('Validation accuracy: %.1f%%' % accuracy(
      #  valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  for i in range(10):
   a = np.argmax(Result.eval())
   print(a) 
  #print(Result.eval())
  #print(weights)
  matplotlib.pyplot.imshow(result)
  matplotlib.pyplot.imshow(weights)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized


NameError: name 'optimizer' is not defined

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

Let's run it:

Initialized
Minibatch loss at step 0: 18.682346
Minibatch accuracy: 7.8%
Validation accuracy: 10.9%
Minibatch loss at step 500: 0.962648
Minibatch accuracy: 82.0%
Validation accuracy: 76.0%
Minibatch loss at step 1000: 1.612884
Minibatch accuracy: 75.0%
Validation accuracy: 76.7%
Minibatch loss at step 1500: 0.712883
Minibatch accuracy: 83.6%
Validation accuracy: 77.5%
Minibatch loss at step 2000: 0.854221
Minibatch accuracy: 82.8%
Validation accuracy: 77.8%
Minibatch loss at step 2500: 0.994794
Minibatch accuracy: 75.8%
Validation accuracy: 78.3%
Minibatch loss at step 3000: 0.968502
Minibatch accuracy: 81.2%
Validation accuracy: 78.5%
[['3.4315903186798096' 'a']
 ['8.652576446533203' 'b']
 ['1.3151963949203491' 'c']
 ['0.6889854669570923' 'd']
 ['7.072232723236084' 'e']
 ['25.346860885620117' 'f']
 ['0.7862876653671265' 'g']
 ['9.341126441955566' 'h']
 ['39.39851379394531' 'i']
 ['3.96663498878479' 'j']]
(10, 2)
i


In [8]:
 img = Image.open('/Users/davidweiss/Documents/Processing/MachineLearn/letter.png')
 img = img.convert('1')
 #img.show()
 arr = (array(img).astype(float)) - 1/2
 arr = np.reshape(arr, 28*28,-1)
 print(arr.shape)
 #print(arr)
 #print(img)
 #a = Image.fromarray(arr,'RGB')
 #a.show()
 arr = np.float32(arr)
 Result = tf.nn.softmax(tf.matmul(arr, weights) + biases)
 print(weights)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/davidweiss/Documents/Processing/MachineLearn/letter.png'